In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 13.6 MB/s eta 0:00:00


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import cv2 as cv

import inspect
path = "/content/drive/Shareddrives/IA DL_project/ML IA/IMAGE_PROCESSING_PIPELINE"

if path not in sys.path:
  sys.path.append(path)

import luna_module
from luna_module import *

# List all function names in the luna_module
function_names = [name for name, obj in inspect.getmembers(luna_module) if inspect.isfunction(obj)]
print(function_names)

['annotations_by_uid', 'binarize_lung', 'binarize_lung_3d', 'binary_closing', 'binary_dilation', 'binary_erosion', 'binary_fill_holes', 'binary_opening', 'center_of_mass', 'clear_border', 'convert_annotation_df', 'convert_annotation_df_with_uid', 'create_3d_mask', 'create_annotations_mask', 'create_annotations_mask_by_uid', 'create_patch', 'debugger', 'draw_ellipsoid', 'find_by_uid', 'find_neighborhood_indices', 'find_neighborhood_indices_more_precise', 'get_slice_candidates', 'get_slice_candidates_old', 'get_slices', 'get_uids', 'img_by_uid', 'masked_annotations_by_uid', 'masked_annotations_with_info_by_uid', 'meta_by_uid', 'norm2float', 'norm2uint16', 'norm2uint8', 'normalize_intensity', 'plot_slices', 'process_slice_candidates', 'process_slices', 'remove_non_central_objects', 'sensitivity_score', 'sensitivity_score_more_precise', 'show_3_images', 'subset_by_uid', 'unwanted_object_filter']


In [ ]:
LUNA_PATH = os.path.join(os.getcwd(), "drive", "Shareddrives", "IA DL_project", "ML IA", "LUNA16")

SUBSET = "subset8"

SUBSETS_PATH = os.path.join(LUNA_PATH, "candidates", "solidity_on_z") # path for subsets folder
CANDIDATES_CSV_PATH =  os.path.join(SUBSETS_PATH, SUBSET, f"{SUBSET}_candidates.csv") # path for subsets folder with y filtered candidates
# SAVE_FOLDER_PATH = os.path.join(LUNA_PATH, "feature_extractions", "pretrained_gabors")


CANDIDATES_DF = pd.read_csv(CANDIDATES_CSV_PATH, index_col="Unnamed: 0")

FILENAMES = os.listdir(os.path.join(SUBSETS_PATH, SUBSET, "y"))
UIDS = set(map(lambda filename: os.path.splitext(filename)[0], FILENAMES))

In [ ]:
CANDIDATES_DF

,seriesuid,x,y,z,Class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,42,303,26,0.0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,471,291,27,0.0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,475,283,28,0.0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,216,312,28,0.0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,476,279,29,0.0
...,...,...,...,...,...
61117,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,284,249,258,0.0
61118,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,289,250,259,0.0
61119,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,293,277,262,0.0
61120,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,297,263,263,0.0


# FEATURE FUNCTIONS

## HoG

In [ ]:
from skimage.feature import hog
from skimage import exposure

def extract_hog_features(patch):
    """
    Extract Histogram of Oriented Gradients (HOG) features from a given patch.

    Args:
        patch (np.ndarray): The input image patch (64x64).

    Returns:
        tuple: HOG features and HOG image.
    """
    # Ensure the patch is of type float32
    patch = patch.astype(np.float32)

    # Compute HOG features
    hog_features = hog(patch,
                        orientations=9,
                        pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2),
                        visualize=False)

    return hog_features

## ALL FEATURES

In [ ]:
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.measure import regionprops, label
from skimage import io, color


def extract_features(nodule_roi,  z_values, thresholding="otsu", verbose=False, debug=False):
  z_max ,z_i = z_values[0], z_values[1]

  # Distance from z-boundary
  z_distance_from_boundary = min(z_i, z_max-z_i)
  if debug:
    print(f"z_distance_from_boundary: {z_distance_from_boundary}")

  hog_features = hog(nodule_roi)

  # Aggregate features into a dictionary or array
  feature_dict = {
      'z_distance_from_boundary': z_distance_from_boundary,
      'z_distance_from_boundary_percent': (z_distance_from_boundary / z_max),
      'hog': hog_features.tolist()
  }


  # Unraveling/unpacking keys with lists in them
  expanded_features = {}

  # Iterate over the original dictionary
  for key, value in feature_dict.items():
      if isinstance(value, list) and all(isinstance(i, list) for i in value):
          # If the value is a list of lists, unpack the sublists into new keys
          for i, sublist in enumerate(value, start=1):
              for j, v in enumerate(sublist, start=1):
                  new_key = f"{key}{i}{j}"
                  expanded_features[new_key] = v
      elif isinstance(value, list):
          # If the value is a list, unpack the list into new keys
          for i, v in enumerate(value, start=1):
              new_key = f"{key}{i}"
              expanded_features[new_key] = v
      else:
          # If the value is not a list of lists, keep the original key-value pair
          expanded_features[key] = value

  return expanded_features

# LOOP THORUGH SUBSET

In [ ]:
CANDIDATES_DF[CANDIDATES_DF["Class"]!=0]

,seriesuid,x,y,z,Class
249,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,45,212,78,1.0
1038,1.3.6.1.4.1.14519.5.2.1.6279.6001.131939324905...,169,365,42,1.0
1387,1.3.6.1.4.1.14519.5.2.1.6279.6001.131939324905...,125,371,82,1.0
2227,1.3.6.1.4.1.14519.5.2.1.6279.6001.137773550852...,417,197,193,1.0
2353,1.3.6.1.4.1.14519.5.2.1.6279.6001.137773550852...,367,232,205,1.0
...,...,...,...,...,...
60454,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,225,288,135,1.0
60457,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,225,310,137,1.0
60458,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,226,315,136,1.0
60491,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,206,288,139,1.0


In [ ]:
subset_candidate_features = pd.DataFrame()
nodules = {}

for i, uid in enumerate(UIDS):
  print(f"\n[START] - {uid} - case {i+1} out of {len(UIDS)}")

  candidates_for_uid_df = annotations_by_uid(uid, CANDIDATES_DF)
  #print(candidates_for_uid_df)

  print(f"[READIN IMG] - {uid}")

  img_path = os.path.join(LUNA_PATH, "subsets", SUBSET, f"{uid}.mhd") # .mhd path
  mhd_img = sitk.ReadImage(img_path) # SimpleITK object
  img_3d = sitk.GetArrayFromImage(mhd_img)
  img_3d[img_3d < AIR_TH] = AIR_TH # Normalize minimum intensity value

  # EXTRACT FEATURES
  candidate_features_df = pd.DataFrame()

  print(f"[FEATURES EXTRACTION START] - {uid} - extracting features for {len(candidates_for_uid_df)} candidates")
  for i, row in candidates_for_uid_df.iterrows():
    debug = False

    slice_ = img_3d[row["z"],:,:] # get slice for candidate
    candidate_roi, _ = create_patch(slice_, (row["y"],row["x"]))

    if row["Class"]:
      debug = True
      nodules[i] = {"uid": uid, "patch": candidate_roi, }

    if debug:
      debugger(candidate_roi, "candidate_roi")

    try:
      fts = extract_features(candidate_roi,  z_values=(img_3d.shape[0], row["z"]), debug=debug)

      # print(f"Features:")
      # print(fts)

      features_df = pd.DataFrame([fts], index=[i])
      print(features_df)
      candidate_features_df = pd.concat([candidate_features_df, features_df])
    except Exception as e: # If feature extraction fails (likely empty image) remove the candidate
      print(f"[ERROR] - {e}")
      print(f"[INFO] - candidate #{i}")
      continue
  print(f"[FEATURES EXTRACTION END] - {uid} - extracted features for {len(candidate_features_df)} candidates\n")

  subset_candidate_features = pd.concat([subset_candidate_features, candidate_features_df])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
SAVE_FOLDER_PATH = os.path.join("/content/drive/Shareddrives/IA DL_project/ML IA/LUNA16/feature_extractions/pretrained_gabors", SUBSET, f"hog_features.csv")
subset_candidate_features.to_csv(SAVE_FOLDER_PATH)

subset_candidate_features

,z_distance_from_boundary,z_distance_from_boundary_percent,hog1,hog2,hog3,hog4,hog5,hog6,hog7,hog8,...,hog1287,hog1288,hog1289,hog1290,hog1291,hog1292,hog1293,hog1294,hog1295,hog1296
6429,23,0.182540,0.056130,0.092104,0.019201,0.035292,0.073981,0.039561,0.053142,0.036050,...,0.004696,0.014757,0.003299,0.000000,0.008934,0.107946,0.257622,0.286658,0.017513,0.005141
6430,24,0.190476,0.126275,0.038272,0.026783,0.006663,0.040945,0.027034,0.070940,0.097591,...,0.014701,0.086161,0.001986,0.031805,0.000000,0.215436,0.307655,0.204862,0.024204,0.016568
6431,25,0.198413,0.064515,0.077602,0.036763,0.080192,0.068621,0.040519,0.053803,0.084706,...,0.047316,0.030208,0.024774,0.018249,0.009432,0.110789,0.079987,0.325294,0.103614,0.003180
6432,26,0.206349,0.098568,0.082044,0.044383,0.033001,0.081622,0.052946,0.078573,0.046277,...,0.021888,0.076540,0.019047,0.022590,0.016959,0.033246,0.065232,0.330823,0.176952,0.007688
6433,27,0.214286,0.068979,0.063495,0.023115,0.024810,0.046107,0.038356,0.063590,0.012463,...,0.014455,0.106505,0.000000,0.009134,0.005451,0.025090,0.037499,0.211481,0.135329,0.007120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43128,17,0.153153,0.190938,0.165046,0.105779,0.035048,0.027161,0.009374,0.000000,0.090447,...,0.052337,0.191220,0.018358,0.014226,0.010757,0.049581,0.021371,0.013933,0.098062,0.202513
43129,17,0.153153,0.040044,0.033166,0.024118,0.088429,0.088074,0.053150,0.041958,0.030917,...,0.168645,0.012464,0.000000,0.006291,0.103419,0.178519,0.061745,0.000000,0.000000,0.002219
43130,17,0.153153,0.169131,0.106168,0.065493,0.076414,0.114960,0.020187,0.021760,0.140281,...,0.084521,0.229955,0.168523,0.137134,0.146731,0.170030,0.146679,0.150911,0.118699,0.182178
43131,13,0.117117,0.078725,0.010215,0.004645,0.012712,0.045181,0.034929,0.040441,0.121690,...,0.015344,0.104735,0.025053,0.007514,0.019579,0.250594,0.142023,0.016799,0.010020,0.005362
